Imports

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv() # load variables from .env file

False

Load File

In [2]:
queries_file = "/Users/hannahzhang/Desktop/Github Repos/ERSP-TeamYang/data/collection.tsv" # store file containing queries

Convert to df for pre-LLM analysis

In [4]:
df = pd.read_csv(queries_file, sep="\t", names = ['query'])

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [25]:
for i in range(5):
    print(df['query'].iloc[i])

The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.
The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.
Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this project would forever change the world forever making it known that something this powerful can be manmade.
The Manhattan Project was the name for a project conducted during World War II, to develop the first atomic bomb. It refers specifically to the period of the project from 194 â¦ 2-1946 under the control of the U.S. Army Corps of Engineers, under 

Feed file to Chatgpt for query classification

- Because Chatgpt has a token processing limit, the tsv file has to be split up (total # queries is 8841823). The below implementation gives Chatgpt subsets of 300 queries, asks for category names, and appends to categories_list. The idea was to take the intersection of all category names provided and then provide those category names to Chatgpt as a template for classifying the queries themselves.

- However, the current prompt for category names is too loose and the category names are too specific. Could feed back complete list of categories given and ask Chatgpt to summarize them into more general categories.

- Should look into engineering a better prompt for this task.

- Current number of subsets is 8842

In [26]:
classification_prompt = '''Please return a list of category names that you would classify these queries into provided the following queries. 
                           Only give me category names, do not return anything else.'''

In [28]:
subset_size = 1000
subset = []
categories_list = []

client = OpenAI()

with open(queries_file, "r") as file:
    for line_number, line in enumerate(file, start=1):
        subset.append(line)

        if len(subset) % subset_size == 0:
            subset_content = "".join(subset)

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "user", "content": f"{classification_prompt}:\n{subset_content}"}
                ]
            )

            print(response.choices[0].message)
            categories_list.append(response.choices[0].message)

            subset = []

if subset:
    subset_content = "".join(subset)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": f"Analyze this dataset chunk:\n{subset_content}"}
        ]
    )
    print(response['choices'][0]['message']['content'])

ChatCompletionMessage(content='1. History  \n2. Science  \n3. Criminal Justice  \n4. Botany  \n5. Immigration  \n6. Medicine/Healthcare  \n7. Music  \n8. Real Estate  \n9. Environmental Science  \n10. Economics  \n11. Sports  \n12. Agriculture  \n13. Sociology  \n14. Physics  \n15. Chemistry  \n16. Law  \n17. Technology  \n18. Psychology  \n19. Genealogy  \n20. Literature  \n21. Cooking  \n22. Pop Culture  \n23. Wildlife  \n24. Fabric  \n25. Education  \n26. Politics  \n27. Transportation  \n28. Personal Finance  \n29. Insurance  \n30. Telecommunications  \n31. Marketing  \n32. Marine Biology  \n33. Automotive  \n34. Travel  \n35. Public Health  \n36. Architecture  \n37. Civil Engineering  \n38. Military  \n39. Art  \n40. Language  \n41. Sports Medicine  \n42. Fragrance  \n43. Event Planning  \n44. Sports History  \n45. Celebrity  \n46. Military History  \n47. Social Issues  \n48. Home Improvement  \n49. Simulation  \n50. Digital Media  \n51. Renewable Energy  \n52. Mythology  \n53. Pe

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-BlqT48soynRWQSo7BpBl2H4N on tokens per min (TPM): Limit 200000, Used 182560, Requested 82927. Please try again in 19.646s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [39]:
print(categories_list)

[]


In [ ]:
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", 
         "content": "You are a helpful data classification assistant."},
        {
            "role": "user",
            "content": f"Provided is the tsv file containing the queries: \n{content}. {classification_prompt}"
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Function calls itself,  \nLayers of thought intertwine,  \nEchoes in the code.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
